<a href="https://colab.research.google.com/github/mberikov/mberikov-OptimizationSIB200/blob/main/sib200_ru_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
!pip install evaluate

In [ ]:
from typing import List, Tuple

In [ ]:
import os
os.environ['WANDB_DISABLED'] = 'true'
from datasets import load_dataset, Dataset
import evaluate
import numpy as np
from sklearn.metrics import classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import pipeline

In [ ]:
MODEL_NAME = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

In [ ]:
print(train_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 701
})


In [ ]:
MINIBATCH_SIZE = 64
tokenized_train_set = train_set.map(
    lambda it: tokenizer(it['text'], truncation=True),
    batched=True, batch_size=MINIBATCH_SIZE
)
tokenized_validation_set = validation_set.map(
    lambda it: tokenizer(it['text'], truncation=True),
    batched=True, batch_size=MINIBATCH_SIZE
)

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
print(tokenized_train_set)

Dataset({
    features: ['index_id', 'category', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 701
})


In [ ]:
cls_metric = evaluate.load('f1')

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return cls_metric.compute(predictions=predictions, references=labels, average='macro')

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
indices_of_categories = list(range(len(list_of_categories)))
n_categories = len(list_of_categories)
print(f'Categories for classification are: {list_of_categories}')
id2label = dict(zip(indices_of_categories, list_of_categories))
label2id = dict(zip(list_of_categories, indices_of_categories))

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [ ]:
labeled_train_set = tokenized_train_set.add_column(
    'label',
    [label2id[val] for val in tokenized_train_set['category']]
)
labeled_validation_set = tokenized_validation_set.add_column(
    'label',
    [label2id[val] for val in tokenized_validation_set['category']]
)

In [ ]:
print(labeled_train_set)

Dataset({
    features: ['index_id', 'category', 'text', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 701
})


In [ ]:
classifier = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=n_categories, id2label=id2label, label2id=label2id
).cuda()
for param in classifier.parameters(): param.data = param.data.contiguous()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='rubert_sib200',
    learning_rate=2e-5,
    per_device_train_batch_size=MINIBATCH_SIZE,
    per_device_eval_batch_size=MINIBATCH_SIZE,
    num_train_epochs=20,
    weight_decay=1e-3,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
training_args = TrainingArguments(
    output_dir='rubert_sib200',
    learning_rate=1e-5,
    per_device_train_batch_size=MINIBATCH_SIZE,
    per_device_eval_batch_size=MINIBATCH_SIZE,
    num_train_epochs=30,
    weight_decay=1e-3,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    fp16=True,
    save_total_limit=3,
    gradient_accumulation_steps=2
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model=classifier,
    args=training_args,
    train_dataset=labeled_train_set,
    eval_dataset=labeled_validation_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
0,No log,1.889609,0.087685
2,No log,1.620660,0.310724
4,No log,1.417091,0.432560
6,No log,1.226356,0.564626
8,No log,1.019319,0.760184
10,No log,0.840004,0.806955
12,No log,0.711751,0.825647
14,No log,0.616734,0.837338
16,No log,0.564835,0.837338
18,No log,0.522748,0.848096


TrainOutput(global_step=150, training_loss=0.7771616617838542, metrics={'train_runtime': 639.8662, 'train_samples_per_second': 32.866, 'train_steps_per_second': 0.234, 'total_flos': 576673334681250.0, 'train_loss': 0.7771616617838542, 'epoch': 27.272727272727273})

In [ ]:
classifiсation_pipeline = pipeline('text-classification', model=classifier, tokenizer=tokenizer, device=0)

In [ ]:
y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(validation_set['text'])))
y_true = validation_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

                    precision    recall  f1-score   support

     entertainment       0.83      0.56      0.67         9
         geography       0.75      0.75      0.75         8
            health       1.00      0.82      0.90        11
          politics       0.93      0.93      0.93        14
science/technology       0.86      0.96      0.91        25
            sports       1.00      0.92      0.96        12
            travel       0.65      0.75      0.70        20

          accuracy                           0.84        99
         macro avg       0.86      0.81      0.83        99
      weighted avg       0.85      0.84      0.84        99



In [ ]:
y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(test_set['text'])))
y_true = test_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

                    precision    recall  f1-score   support

     entertainment       0.92      0.58      0.71        19
         geography       0.89      1.00      0.94        17
            health       0.95      0.86      0.90        22
          politics       0.97      0.97      0.97        30
science/technology       0.89      0.94      0.91        51
            sports       0.96      0.96      0.96        25
            travel       0.89      0.97      0.93        40

          accuracy                           0.92       204
         macro avg       0.92      0.90      0.90       204
      weighted avg       0.92      0.92      0.91       204



Работал с гиперпараметрами, ипсользовал 1)learning rate: чтобы снизить скорость обучения (для более стабильности и оптимальности), 2) num-train-epochs - увеличил кол-во эпох, чтобы увеличить качество модели, 3) fp16=True,  - ускорение обучение на GPU, 4) Save_total_limit - ограничение на макс.кол-во сохранения контр. точек, чтобы сэкономить место на диске (экономия места на диске)